In [1]:
import pandas as pd
import numpy as np

#libraries for web scrapping
import urllib.request
from bs4 import BeautifulSoup

# for latitude and longitude of neighborhoods
!pip install geocoder
import geocoder 

import requests #foursqaure API requests
from pandas.io.json import json_normalize #get data from json file

from sklearn.cluster import KMeans #kmeans clustering

#libraries for map
import matplotlib.cm as cm
import matplotlib.colors as colors
import folium 
from geopy.geocoders import Nominatim #for latitude and longitude of city

     |████████████████████████████████| 98 kB 2.6 MB/s eta 0:00:011


Web Scrapping

In [2]:
url = 'https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M'

In [3]:
page = urllib.request.urlopen(url)
soup = BeautifulSoup(page, "lxml")

In [4]:
#find right table
all_tables=soup.find_all("table")
all_tables
right_table=soup.find('table', class_='wikitable sortable')

In [5]:
#get data in columns
A = []
B = []
C = []

for row in right_table.findAll('tr'):
    cells=row.findAll('td')
    if len(cells)==3:
        A.append(cells[0].find(text=True))
        B.append(cells[1].find(text=True))
        C.append(cells[2].find(text=True))

Dataframe Wrangling

In [6]:
#convert to pandas dataframe
df=pd.DataFrame(A,columns=['PostalCode'])
df['Borough']=B
df['Neighborhood']=C
df

,PostalCode,Borough,Neighborhood
0,M1A\n,Not assigned\n,\n
1,M2A\n,Not assigned\n,\n
2,M3A\n,North York\n,Parkwoods\n
3,M4A\n,North York\n,Victoria Village\n
4,M5A\n,Downtown Toronto\n,Regent Park / Harbourfront\n
...,...,...,...
175,M5Z\n,Not assigned\n,\n
176,M6Z\n,Not assigned\n,\n
177,M7Z\n,Not assigned\n,\n
178,M8Z\n,Etobicoke\n,Mimico NW / The Queensway West / South of Bloo...


In [7]:
#remove extra strings
df['PostalCode'] = df['PostalCode'].str[:-1]
df['Borough'] = df['Borough'].str[:-1]
df['Neighborhood'] = df['Neighborhood'].str[:-1]

In [8]:
df

,PostalCode,Borough,Neighborhood
0,M1A,Not assigned,
1,M2A,Not assigned,
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,Regent Park / Harbourfront
...,...,...,...
175,M5Z,Not assigned,
176,M6Z,Not assigned,
177,M7Z,Not assigned,
178,M8Z,Etobicoke,Mimico NW / The Queensway West / South of Bloo...


In [9]:
#filter out not assigned
df = df[~df.Borough.str.contains("Not assigned")]

In [10]:
#check neighborhood len and 'Not assigned'
print(df[df['Neighborhood'].str.len()==0])
print(df[df.Neighborhood.str.contains('Not assigned')])

Empty DataFrame
Columns: [PostalCode, Borough, Neighborhood]
Index: []
Empty DataFrame
Columns: [PostalCode, Borough, Neighborhood]
Index: []


In [11]:
#aggregate neighborhood rows into column
df_group = df.groupby(['PostalCode', 'Borough'])['Neighborhood'].apply(','.join).reset_index()
#df_group.Neighborhood.str.replace('/',',')
df_group

,PostalCode,Borough,Neighborhood
0,M1B,Scarborough,Malvern / Rouge
1,M1C,Scarborough,Rouge Hill / Port Union / Highland Creek
2,M1E,Scarborough,Guildwood / Morningside / West Hill
3,M1G,Scarborough,Woburn
4,M1H,Scarborough,Cedarbrae
...,...,...,...
98,M9N,York,Weston
99,M9P,Etobicoke,Westmount
100,M9R,Etobicoke,Kingsview Village / St. Phillips / Martin Grov...
101,M9V,Etobicoke,South Steeles / Silverstone / Humbergate / Jam...


In [12]:
df_group.shape

(103, 3)

Getting longitude & latitude of postal codes

In [13]:
#define function for lat long fetching
def get_latlon(postal_code):
    lat_lng_coords = None
    while(lat_lng_coords is None):
        g = geocoder.arcgis('{}, Toronto, Ontario'.format(postal_code))
        lat_lng_coords = g.latlng
        return lat_lng_coords 



In [14]:
#fetching lat long data
latlog = []
    
for i in df_group['PostalCode']:
    a = get_latlon(i)
    latlog.append(a)
    
#df_group['latitude'] = df_group['PostalCode'].apply(get_latlon)

In [15]:
#adding lat long columns to dataframe
latlog = np.asarray(latlog)
df_group['Latitude'] = latlog[:,0]
df_group['Longitude'] = latlog[:,1]
#df_group.drop(['latitude','longitude'],axis=1,inplace=True)

In [16]:
df_group

,PostalCode,Borough,Neighborhood,Latitude,Longitude
0,M1B,Scarborough,Malvern / Rouge,43.808626,-79.189913
1,M1C,Scarborough,Rouge Hill / Port Union / Highland Creek,43.785779,-79.157368
2,M1E,Scarborough,Guildwood / Morningside / West Hill,43.765806,-79.185284
3,M1G,Scarborough,Woburn,43.771545,-79.218135
4,M1H,Scarborough,Cedarbrae,43.768791,-79.238813
...,...,...,...,...,...
98,M9N,York,Weston,43.705496,-79.520370
99,M9P,Etobicoke,Westmount,43.696296,-79.533126
100,M9R,Etobicoke,Kingsview Village / St. Phillips / Martin Grov...,43.686887,-79.565507
101,M9V,Etobicoke,South Steeles / Silverstone / Humbergate / Jam...,43.744055,-79.581203


Foursquare API

In [17]:
CLIENT_ID = 'CQ2OUSPP4YDZJSGMF3XC0IEKB01C4KTN3OMJUFMPKCA0LPJ0' # your Foursquare ID
CLIENT_SECRET = 'H5YASFCDHXIQGYQ44W4XO3GJIPOL1R1PU52FPUAXJRCAFVQS' # your Foursquare Secret
VERSION = '20200420' # Foursquare API version


In [20]:
LIMIT = 100 # limit of number of venues returned by Foursquare API
radius = 500 # define radius

In [18]:
#define function to get POIs
def getNearbyVenues(names, latitudes, longitudes, radius=500):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
            
        # create the API request URL
        global url
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighborhood', 
                  'Neighborhood Latitude', 
                  'Neighborhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

In [21]:
#get venue categories for nighborhoods
Toronto_venues = getNearbyVenues(names=df_group['Neighborhood'],#[:14], #temp slicer
                                   latitudes=df_group['Latitude'],
                                   longitudes=df_group['Longitude']
                                  )


Malvern / Rouge
Rouge Hill / Port Union / Highland Creek
Guildwood / Morningside / West Hill
Woburn
Cedarbrae
Scarborough Village
Kennedy Park / Ionview / East Birchmount Park
Golden Mile / Clairlea / Oakridge
Cliffside / Cliffcrest / Scarborough Village West
Birch Cliff / Cliffside West
Dorset Park / Wexford Heights / Scarborough Town Centre
Wexford / Maryvale
Agincourt
Clarks Corners / Tam O'Shanter / Sullivan
Milliken / Agincourt North / Steeles East / L'Amoreaux East
Steeles West / L'Amoreaux West
Upper Rouge
Hillcrest Village
Fairview / Henry Farm / Oriole
Bayview Village
York Mills / Silver Hills
Willowdale / Newtonbrook
Willowdale
York Mills West
Willowdale
Parkwoods
Don Mills
Don Mills
Bathurst Manor / Wilson Heights / Downsview North
Northwood Park / York University
Downsview
Downsview
Downsview
Downsview
Victoria Village
Parkview Hill / Woodbine Gardens
Woodbine Heights
The Beaches
Leaside
Thorncliffe Park
East Toronto
The Danforth West / Riverdale
India Bazaar / The Beaches 

In [22]:
Toronto_venues

,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,Malvern / Rouge,43.808626,-79.189913,Upper Rouge Trail,43.809988,-79.186147,Trail
1,Rouge Hill / Port Union / Highland Creek,43.785779,-79.157368,Fox and Fiddle,43.789082,-79.154459,Bar
2,Guildwood / Morningside / West Hill,43.765806,-79.185284,Chick-N-Joy,43.768752,-79.187982,Fried Chicken Joint
3,Guildwood / Morningside / West Hill,43.765806,-79.185284,Little Caesars Pizza,43.769046,-79.184386,Pizza Place
4,Guildwood / Morningside / West Hill,43.765806,-79.185284,Swiss Chalet,43.768122,-79.190493,Restaurant
...,...,...,...,...,...,...,...
2268,South Steeles / Silverstone / Humbergate / Jam...,43.744055,-79.581203,Caribbean Heat 2,43.743186,-79.582367,Caribbean Restaurant
2269,South Steeles / Silverstone / Humbergate / Jam...,43.744055,-79.581203,Panorama Park,43.747021,-79.583497,Park
2270,South Steeles / Silverstone / Humbergate / Jam...,43.744055,-79.581203,LCBO,43.741508,-79.584501,Liquor Store
2271,South Steeles / Silverstone / Humbergate / Jam...,43.744055,-79.581203,Rogers Plus,43.741312,-79.585263,Video Store


**Prepare for Analysis**

In [23]:
# one hot encoding
Toronto_onehot = pd.get_dummies(Toronto_venues[['Venue Category']], prefix="", prefix_sep="")

# add neighborhood column back to dataframe
Toronto_onehot['Neighborhood'] = Toronto_venues['Neighborhood'] 

# move neighborhood column to the first column
fixed_columns = [Toronto_onehot.columns[-1]] + list(Toronto_onehot.columns[:-1])
Toronto_onehot = Toronto_onehot[fixed_columns]

Toronto_onehot.head()

,Yoga Studio,ATM,Accessories Store,Airport,American Restaurant,Antique Shop,Art Gallery,Art Museum,Arts & Crafts Store,Asian Restaurant,...,Toy / Game Store,Trail,Train Station,Vegetarian / Vegan Restaurant,Video Game Store,Video Store,Vietnamese Restaurant,Wine Bar,Wine Shop,Women's Store
0,0,0,0,0,0,0,0,0,0,0,...,0,1,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [24]:
Toronto_groupby = Toronto_onehot.groupby('Neighborhood').mean().reset_index()
Toronto_groupby

,Neighborhood,Yoga Studio,ATM,Accessories Store,Airport,American Restaurant,Antique Shop,Art Gallery,Art Museum,Arts & Crafts Store,...,Toy / Game Store,Trail,Train Station,Vegetarian / Vegan Restaurant,Video Game Store,Video Store,Vietnamese Restaurant,Wine Bar,Wine Shop,Women's Store
0,Agincourt,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,...,0.0,0.000000,0.0,0.0,0.0,0.00,0.0,0.0,0.0,0.0
1,Alderwood / Long Branch,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,...,0.0,0.000000,0.0,0.0,0.0,0.00,0.0,0.0,0.0,0.0
2,Bathurst Manor / Wilson Heights / Downsview North,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,...,0.0,0.000000,0.0,0.0,0.0,0.05,0.0,0.0,0.0,0.0
3,Bayview Village,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,...,0.0,0.333333,0.0,0.0,0.0,0.00,0.0,0.0,0.0,0.0
4,Bedford Park / Lawrence Manor East,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,...,0.0,0.000000,0.0,0.0,0.0,0.00,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
91,Willowdale / Newtonbrook,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,...,0.0,0.000000,0.0,0.0,0.0,0.00,0.0,0.0,0.0,0.0
92,Woburn,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,...,0.0,0.000000,0.0,0.0,0.0,0.00,0.0,0.0,0.0,0.0
93,Woodbine Heights,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.083333,...,0.0,0.000000,0.0,0.0,0.0,0.00,0.0,0.0,0.0,0.0
94,York Mills / Silver Hills,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,...,0.0,0.000000,0.0,0.0,0.0,0.00,0.0,0.0,0.0,0.0


In [25]:
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

In [26]:
num_top_venues = 10

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['Neighborhood']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# create a new dataframe
neighborhoods_venues_sorted = pd.DataFrame(columns=columns)
neighborhoods_venues_sorted['Neighborhood'] = Toronto_groupby['Neighborhood']

for ind in np.arange(Toronto_groupby.shape[0]):
    neighborhoods_venues_sorted.iloc[ind, 1:] = return_most_common_venues(Toronto_groupby.iloc[ind, :], num_top_venues)

neighborhoods_venues_sorted.head()

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Agincourt,Breakfast Spot,Skating Rink,Badminton Court,Sushi Restaurant,Supermarket,Fish & Chips Shop,Fish Market,Field,Fast Food Restaurant,Dumpling Restaurant
1,Alderwood / Long Branch,Sandwich Place,Pizza Place,Convenience Store,Pharmacy,Pub,Gym,Gas Station,Coffee Shop,Eastern European Restaurant,Distribution Center
2,Bathurst Manor / Wilson Heights / Downsview North,Bank,Coffee Shop,Supermarket,Gas Station,Fried Chicken Joint,Restaurant,Diner,Sandwich Place,Deli / Bodega,Sushi Restaurant
3,Bayview Village,Trail,Construction & Landscaping,Flower Shop,Farmers Market,Eastern European Restaurant,Electronics Store,Ethiopian Restaurant,Falafel Restaurant,Farm,Women's Store
4,Bedford Park / Lawrence Manor East,Coffee Shop,Italian Restaurant,Sandwich Place,Café,Juice Bar,Restaurant,Sports Club,Thai Restaurant,Pub,Indian Restaurant


Clustering

In [27]:
# set number of clusters
kclusters = 5

Toronto_clustering = Toronto_groupby.drop('Neighborhood', 1)

# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(Toronto_clustering)

# check cluster labels generated for each row in the dataframe
kmeans.labels_[0:10] 

array([0, 0, 0, 2, 0, 0, 0, 0, 0, 0], dtype=int32)

In [31]:
neighborhoods_venues_sorted.insert(0, 'Cluster Labels', kmeans.labels_)
Toronto_merged = df_group#[:14] #temp slicer
Toronto_merged = Toronto_merged.join(neighborhoods_venues_sorted.set_index('Neighborhood'), on='Neighborhood')


Toronto_merged.head() 

,PostalCode,Borough,Neighborhood,Latitude,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,M1B,Scarborough,Malvern / Rouge,43.808626,-79.189913,2.0,Trail,Farmers Market,Dumpling Restaurant,Eastern European Restaurant,Electronics Store,Ethiopian Restaurant,Falafel Restaurant,Farm,Women's Store,Dog Run
1,M1C,Scarborough,Rouge Hill / Port Union / Highland Creek,43.785779,-79.157368,0.0,Bar,Women's Store,Donut Shop,Food,Flower Shop,Fish Market,Fish & Chips Shop,Field,Fast Food Restaurant,Farmers Market
2,M1E,Scarborough,Guildwood / Morningside / West Hill,43.765806,-79.185284,0.0,Pizza Place,Fast Food Restaurant,Breakfast Spot,Intersection,Coffee Shop,Restaurant,Salon / Barbershop,Mexican Restaurant,Bus Station,Sports Bar
3,M1G,Scarborough,Woburn,43.771545,-79.218135,0.0,Coffee Shop,Korean Restaurant,Women's Store,Eastern European Restaurant,Electronics Store,Ethiopian Restaurant,Falafel Restaurant,Farm,Farmers Market,Donut Shop
4,M1H,Scarborough,Cedarbrae,43.768791,-79.238813,2.0,Trail,Construction & Landscaping,Fast Food Restaurant,Eastern European Restaurant,Electronics Store,Ethiopian Restaurant,Falafel Restaurant,Farm,Farmers Market,Women's Store


In [42]:
#remove not found neighborhoods & Nan values
Toronto_merged = Toronto_merged[Toronto_merged['Cluster Labels']>0]
Toronto_merged['Cluster Labels'] = Toronto_merged['Cluster Labels'].astype(int)
Toronto_merged = Toronto_merged[~Toronto_merged['Cluster Labels'].isnull()]
Toronto_merged.head()

,PostalCode,Borough,Neighborhood,Latitude,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,M1B,Scarborough,Malvern / Rouge,43.808626,-79.189913,2,Trail,Farmers Market,Dumpling Restaurant,Eastern European Restaurant,Electronics Store,Ethiopian Restaurant,Falafel Restaurant,Farm,Women's Store,Dog Run
4,M1H,Scarborough,Cedarbrae,43.768791,-79.238813,2,Trail,Construction & Landscaping,Fast Food Restaurant,Eastern European Restaurant,Electronics Store,Ethiopian Restaurant,Falafel Restaurant,Farm,Farmers Market,Women's Store
14,M1V,Scarborough,Milliken / Agincourt North / Steeles East / L'...,43.817998,-79.280887,4,Pharmacy,Women's Store,Dog Run,Flower Shop,Fish Market,Fish & Chips Shop,Field,Fast Food Restaurant,Farmers Market,Farm
17,M2H,North York,Hillcrest Village,43.802556,-79.356566,3,Dog Run,Residential Building (Apartment / Condo),Park,Donut Shop,Dumpling Restaurant,Eastern European Restaurant,Electronics Store,Ethiopian Restaurant,Women's Store,Distribution Center
19,M2K,North York,Bayview Village,43.780607,-79.376921,2,Trail,Construction & Landscaping,Flower Shop,Farmers Market,Eastern European Restaurant,Electronics Store,Ethiopian Restaurant,Falafel Restaurant,Farm,Women's Store


Visualization

In [43]:
toronto_latitude = '43.651070'
toronto_longitude = '-79.347015'

In [44]:
# create map
map_clusters = folium.Map(location=[toronto_latitude, toronto_longitude], zoom_start=11)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i + x + (i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(Toronto_merged['Latitude'], Toronto_merged['Longitude'], Toronto_merged['Neighborhood'], Toronto_merged['Cluster Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters

**Summary:**

* Cluster 0 - eating out loation with lots of pizzerias and fast food restaurant
* Cluster 1 - this can be regarded as outlier, with just one neighborhood
* Cluster 2 - insteresting neighbothood for outdoor activity
* Cluster 3 - recreational area with parks and pools as top venues 
* Cluster 4 - neighborhoods with no specific concetration of venues


In [45]:
Toronto_merged.loc[Toronto_merged['Cluster Labels'] == 0, Toronto_merged.columns[[2] + list(range(5, Toronto_merged.shape[1]))]]

,Neighborhood,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue


In [46]:
Toronto_merged.loc[Toronto_merged['Cluster Labels'] == 1, Toronto_merged.columns[[2] + list(range(5, Toronto_merged.shape[1]))]]

,Neighborhood,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
102,Northwest,1,Hotel,Women's Store,Farmers Market,Dumpling Restaurant,Eastern European Restaurant,Electronics Store,Ethiopian Restaurant,Falafel Restaurant,Farm,Fast Food Restaurant


In [47]:
Toronto_merged.loc[Toronto_merged['Cluster Labels'] == 2, Toronto_merged.columns[[2] + list(range(5, Toronto_merged.shape[1]))]]

,Neighborhood,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Malvern / Rouge,2,Trail,Farmers Market,Dumpling Restaurant,Eastern European Restaurant,Electronics Store,Ethiopian Restaurant,Falafel Restaurant,Farm,Women's Store,Dog Run
4,Cedarbrae,2,Trail,Construction & Landscaping,Fast Food Restaurant,Eastern European Restaurant,Electronics Store,Ethiopian Restaurant,Falafel Restaurant,Farm,Farmers Market,Women's Store
19,Bayview Village,2,Trail,Construction & Landscaping,Flower Shop,Farmers Market,Eastern European Restaurant,Electronics Store,Ethiopian Restaurant,Falafel Restaurant,Farm,Women's Store
80,Del Ray / Mount Dennis / Keelsdale and Silvert...,2,Construction & Landscaping,Pool,Coffee Shop,Dumpling Restaurant,Eastern European Restaurant,Electronics Store,Ethiopian Restaurant,Falafel Restaurant,Farm,Farmers Market


In [48]:
Toronto_merged.loc[Toronto_merged['Cluster Labels'] == 3, Toronto_merged.columns[[2] + list(range(5, Toronto_merged.shape[1]))]]

,Neighborhood,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
17,Hillcrest Village,3,Dog Run,Residential Building (Apartment / Condo),Park,Donut Shop,Dumpling Restaurant,Eastern European Restaurant,Electronics Store,Ethiopian Restaurant,Women's Store,Distribution Center
23,York Mills West,3,Bank,Convenience Store,Park,Farmers Market,Eastern European Restaurant,Electronics Store,Ethiopian Restaurant,Falafel Restaurant,Farm,Women's Store
25,Parkwoods,3,Food & Drink Shop,Park,Falafel Restaurant,Donut Shop,Dumpling Restaurant,Eastern European Restaurant,Electronics Store,Ethiopian Restaurant,Women's Store,Dog Run
40,East Toronto,3,Playground,Music Venue,Women's Store,Dog Run,Fish Market,Fish & Chips Shop,Field,Fast Food Restaurant,Farmers Market,Farm
41,The Danforth West / Riverdale,3,Business Service,Park,Discount Store,Bus Line,Grocery Store,Eastern European Restaurant,Electronics Store,Ethiopian Restaurant,Falafel Restaurant,Women's Store
45,Davisville North,3,Food & Drink Shop,Breakfast Spot,Department Store,Park,Gym,Dumpling Restaurant,Electronics Store,Ethiopian Restaurant,Falafel Restaurant,Farm
46,North Toronto West,3,Playground,Garden,Gym Pool,Park,Women's Store,Ethiopian Restaurant,Donut Shop,Dumpling Restaurant,Eastern European Restaurant,Electronics Store
48,Moore Park / Summerhill East,3,Playground,Trail,Park,Gym,Women's Store,Falafel Restaurant,Donut Shop,Dumpling Restaurant,Eastern European Restaurant,Electronics Store
50,Rosedale,3,Grocery Store,Playground,Candy Store,Park,Women's Store,Dumpling Restaurant,Eastern European Restaurant,Electronics Store,Ethiopian Restaurant,Falafel Restaurant
59,Harbourfront East / Union Station / Toronto Is...,3,Theme Park,Harbor / Marina,Farm,Park,Women's Store,Ethiopian Restaurant,Donut Shop,Dumpling Restaurant,Eastern European Restaurant,Electronics Store


In [49]:
Toronto_merged.loc[Toronto_merged['Cluster Labels'] == 4, Toronto_merged.columns[[2] + list(range(5, Toronto_merged.shape[1]))]]

,Neighborhood,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
14,Milliken / Agincourt North / Steeles East / L'...,4,Pharmacy,Women's Store,Dog Run,Flower Shop,Fish Market,Fish & Chips Shop,Field,Fast Food Restaurant,Farmers Market,Farm
